## Evolutionary origin

Macaca - Mouse - Human

In [2]:
import os,re,glob
import pandas as pd
import numpy as np
from collections import Counter
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

users_dir = "/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction"
GENOMEDIR = "/genomics/users/marta/genomes"

## annotation file
annotation="/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/human/newReference_Resconstructed/gencode.v38.gffcompare.TestisLiverBrain.annotation.sorted.1transcript.sorted.NOchr.gtf"
transcript_gene=pd.read_csv("/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/human/newReference_Resconstructed/1transcript_1gene.reconstructed.csv")

## evoDir
evoDir=os.path.join(users_dir,"EvolutionaryOrigin")

def translate_dna_to_protein(dna_seq):
    return str(Seq(dna_seq).translate())

# Function to save DF in fasta format
def create_seqrecord_PROT(row):
    return SeqRecord(Seq(row['protein']), id=row['header'][:-4], description="")
def create_seqrecord_DNA(row):
    return SeqRecord(Seq(row['seq']), id=row['header'], description="")



In [43]:
## MACACA - candidates
specie = "macaca"
riboseq_dir = os.path.join(users_dir,specie,"RiboSeq/RiboNovel")
df = pd.read_csv(os.path.join(riboseq_dir,"Annotation/candidateORF.fa"), header=None, sep="\t")

candidates_fasta = pd.DataFrame({'header':df[0].iloc[::2].values, 'seq':df[0].iloc[1::2].values})
candidates_fasta['header'] = candidates_fasta['header'].str[1:]
candidates_fasta


,header,seq
0,ENSMMUT00000032773:1:-|1|2681:41:299|ouORF|ATG,ATGGTCCGGTTCGGGCTCGGGAGGCTCCGTCTGGGCCAGGGCAGGT...
1,ENSMMUT00000032773:1:-|2|2681:71:299|ouORF|CTG,CTGGGCCAGGGCAGGTTCTTAGACTCTGTGAGTAAAGACAGCTTTG...
2,ENSMMUT00000032773:1:-|3|2681:98:299|ouORF|GTG,GTGAGTAAAGACAGCTTTGTCTTCCCAGTTCATCATGGCTTCAACA...
3,ENSMMUT00000032773:1:-|4|2681:114:1911|extensi...,TTGTCTTCCCAGTTCATCATGGCTTCAACATCCAGAGATGTCATTG...
4,ENSMMUT00000032773:1:-|5|2681:132:1911|canonic...,ATGGCTTCAACATCCAGAGATGTCATTGCTGGGAGAGGTATCCACT...
...,...,...
6029294,ENSMMUT00000106277:Y:+|275|4355:4226:4244|dORF...,TTGCCTTTGTTGGAGTGA
6029295,ENSMMUT00000106277:Y:+|276|4355:4232:4244|dORF...,TTGTTGGAGTGA
6029296,ENSMMUT00000106277:Y:+|277|4355:4240:4261|dORF...,GTGAAGTATTTAAAAGAGTAA
6029297,ENSMMUT00000106277:Y:+|278|4355:4268:4283|dORF...,GTGATAAAGTCCTGA


In [47]:
## get repre sequences - both DNA and PROTEINS
specie = "macaca"
riboseq_dir = os.path.join(users_dir,specie,"RiboSeq/RiboNovel")
all_repre_candidates = pd.DataFrame()
for sample in os.listdir(os.path.join(riboseq_dir,"RibORF")):
    if sample.endswith("r1"):
        print(sample)
        repre = pd.read_csv(os.path.join(riboseq_dir,"RibORF",sample,"repre.valid.pred.pvalue.parameters.txt"), sep="\t")
        repre_fasta = candidates_fasta[candidates_fasta['header'].isin(repre.orfID.values.tolist())]
        repre_fasta['protein'] = repre_fasta['seq'].apply(translate_dna_to_protein)
        all_repre_candidates = pd.concat([all_repre_candidates, repre_fasta])
        out_proteins = os.path.join(riboseq_dir,"RibORF",sample,"repre.valid.pred.pvalue.parameters.PROTEIN.fa")
        seq_records_proteins = repre_fasta.apply(create_seqrecord_PROT, axis=1).tolist()
        SeqIO.write(seq_records_proteins, out_proteins, "fasta")

        out_dna = os.path.join(riboseq_dir,"RibORF",sample,"repre.valid.pred.pvalue.parameters.DNA.fa")
        seq_records_dna = repre_fasta.apply(create_seqrecord_DNA, axis=1).tolist()
        SeqIO.write(seq_records_dna, out_dna, "fasta")


macaque_brain_ribo_3_r1


/tmp/ipykernel_880173/3428765972.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repre_fasta['protein'] = repre_fasta['seq'].apply(translate_dna_to_protein)


macaque_brain_ribo_2_r1


/tmp/ipykernel_880173/3428765972.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repre_fasta['protein'] = repre_fasta['seq'].apply(translate_dna_to_protein)


macaque_brain_ribo_1_r1


...

/tmp/ipykernel_880173/3428765972.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repre_fasta['protein'] = repre_fasta['seq'].apply(translate_dna_to_protein)


macaque_liver_ribo_2_r1


/tmp/ipykernel_880173/3428765972.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repre_fasta['protein'] = repre_fasta['seq'].apply(translate_dna_to_protein)


macaque_liver_ribo_3_r1


/tmp/ipykernel_880173/3428765972.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repre_fasta['protein'] = repre_fasta['seq'].apply(translate_dna_to_protein)


In [49]:
all_repre_candidates.drop_duplicates(inplace=True)
all_repre_candidates
out_nonredundant_proteins = os.path.join(riboseq_dir,"RibORF/repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa")
seq_records_proteins = all_repre_candidates.apply(create_seqrecord_PROT, axis=1).tolist()
SeqIO.write(seq_records_proteins, out_nonredundant_proteins, "fasta")

34916

In [50]:
## MOUSE - candidates
specie = "mouse"
riboseq_dir = os.path.join(users_dir,specie,"RiboSeq/RiboNovel")
df = pd.read_csv(os.path.join(riboseq_dir,"Annotation/candidateORF.fa"), header=None, sep="\t")

candidates_fasta = pd.DataFrame({'header':df[0].iloc[::2].values, 'seq':df[0].iloc[1::2].values})
candidates_fasta['header'] = candidates_fasta['header'].str[1:]
candidates_fasta


,header,seq
0,ENSMUST00000082387.1:chrM:+|1|68:45:66|noncodi...,ATGGATAATTGTATCCCATAA
1,ENSMUST00000082388.1:chrM:+|1|955:7:25|noncodi...,TTGGTCCTGGCCTTATAA
2,ENSMUST00000082388.1:chrM:+|2|955:13:25|noncod...,CTGGCCTTATAA
3,ENSMUST00000082388.1:chrM:+|3|955:47:83|noncod...,ATGCAAACCTCCATAGACCGGTGTAAAATCCCTTAA
4,ENSMUST00000082388.1:chrM:+|4|955:141:186|nonc...,TTGCCTAGCCACACCCCCACGGGACTCAGCAGTGATAAATATTAA
...,...,...
7875841,TCONS_00000047:MU069435.1:-|5|345:57:288|nonco...,TTGTCACCACTAGGTGTCGCCCGACAATGGTCTCGATCAACCATAC...
7875842,TCONS_00000047:MU069435.1:-|6|345:70:274|nonco...,GTGTCGCCCGACAATGGTCTCGATCAACCATACAAACCAATACAGA...
7875843,TCONS_00000047:MU069435.1:-|7|345:83:329|nonco...,ATGGTCTCGATCAACCATACAAACCAATACAGATCGCTCACGCGCC...
7875844,TCONS_00000047:MU069435.1:-|8|345:149:329|nonc...,CTGTCTCTCCGGAGGGTCTCCCGAGCTCAGGTTCCGGCACTCACGC...


In [51]:
## get repre sequences - both DNA and PROTEINS
specie = "mouse"
riboseq_dir = os.path.join(users_dir,specie,"RiboSeq/RiboNovel")
all_repre_candidates = pd.DataFrame()
for sample in os.listdir(os.path.join(riboseq_dir,"RibORF")):
    if sample.endswith("r1"):
        print(sample)
        repre = pd.read_csv(os.path.join(riboseq_dir,"RibORF",sample,"repre.valid.pred.pvalue.parameters.txt"), sep="\t")
        repre_fasta = candidates_fasta[candidates_fasta['header'].isin(repre.orfID.values.tolist())]
        repre_fasta['protein'] = repre_fasta['seq'].apply(translate_dna_to_protein)
        all_repre_candidates = pd.concat([all_repre_candidates, repre_fasta])
        out_proteins = os.path.join(riboseq_dir,"RibORF",sample,"repre.valid.pred.pvalue.parameters.PROTEIN.fa")
        seq_records_proteins = repre_fasta.apply(create_seqrecord_PROT, axis=1).tolist()
        SeqIO.write(seq_records_proteins, out_proteins, "fasta")

        out_dna = os.path.join(riboseq_dir,"RibORF",sample,"repre.valid.pred.pvalue.parameters.DNA.fa")
        seq_records_dna = repre_fasta.apply(create_seqrecord_DNA, axis=1).tolist()
        SeqIO.write(seq_records_dna, out_dna, "fasta")


mouse_spermatozoa_ribo_2_r1


/tmp/ipykernel_880173/1259375188.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repre_fasta['protein'] = repre_fasta['seq'].apply(translate_dna_to_protein)


mouse_spermatocytes_ribo_2_r1


/tmp/ipykernel_880173/1259375188.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repre_fasta['protein'] = repre_fasta['seq'].apply(translate_dna_to_protein)


mouse_testis_ribo_2_r1


...

/tmp/ipykernel_880173/1259375188.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repre_fasta['protein'] = repre_fasta['seq'].apply(translate_dna_to_protein)


mouse_elongatingSpermatids_ribo_1_r1


/tmp/ipykernel_880173/1259375188.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repre_fasta['protein'] = repre_fasta['seq'].apply(translate_dna_to_protein)


mouse_roundSpermatids_ribo_2_r1


/tmp/ipykernel_880173/1259375188.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repre_fasta['protein'] = repre_fasta['seq'].apply(translate_dna_to_protein)


In [52]:
all_repre_candidates.drop_duplicates(inplace=True)
all_repre_candidates
out_nonredundant_proteins = os.path.join(riboseq_dir,"RibORF/repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa")
seq_records_proteins = all_repre_candidates.apply(create_seqrecord_PROT, axis=1).tolist()
SeqIO.write(seq_records_proteins, out_nonredundant_proteins, "fasta")

54418

## BLASTP testis human

Against all expressed in mouse and macaca in any tissue (non-redunant set)

#### TESTIS-EXPRESSED

In [48]:
## Testis-expressed ORFs human
TestisExpressed_translatedINtestis = pd.read_csv("/projects_eg/projects/marta/TestisRestricted_Microproteins_TSA/with_TranscriptomeReconstruction/ribORF_humanTestis_in2.csv")
TestisExpressed_translatedINtestis

def create_seqrecord_PROThuman(row):
    return SeqRecord(Seq(row['ORFpep']), id=row['orfID'], description="")

out_nonredundant_proteins = "/projects_eg/projects/marta/TestisRestricted_Microproteins_TSA/with_TranscriptomeReconstruction/ribORF_humanTestis_in2.fa"
seq_records_proteins = TestisExpressed_translatedINtestis.apply(create_seqrecord_PROThuman, axis=1).tolist()
SeqIO.write(seq_records_proteins, out_nonredundant_proteins, "fasta")

12866

In [65]:
%%bash -s "$evoDir"

echo $1
## create blastp database
module load BLAST+/2.12.0-Linux_x86_64

## macaca
awk 'BEGIN{FS=OFS="|"} /^>/ { $3 = ""; sub(/\|\|/, "|") } { print }' /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/macaca/RiboSeq/RiboNovel/RibORF/repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa > /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/macaca/RiboSeq/RiboNovel/RibORF/repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.shortheaders.fa
makeblastdb -in /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/macaca/RiboSeq/RiboNovel/RibORF/repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.shortheaders.fa -out $1/macaca_db_prot -parse_seqids -dbtype prot
## mouse
awk 'BEGIN{FS=OFS="|"} /^>/ { $3 = ""; sub(/\|\|/, "|") } { print }' /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/mouse/RiboSeq/RiboNovel/RibORF/repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.fa > /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/mouse/RiboSeq/RiboNovel/RibORF/repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.shortheaders.fa
makeblastdb -in /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/mouse/RiboSeq/RiboNovel/RibORF/repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.shortheaders.fa -out $1/mouse_db_prot -parse_seqids -dbtype prot


/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/EvolutionaryOrigin


Building a new DB, current time: 07/16/2024 12:02:23
New DB name:   /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/EvolutionaryOrigin/mouse_db_prot
New DB title:  /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/mouse/RiboSeq/RiboNovel/RibORF/repre.valid.pred.pvalue.parameters.PROTEIN.noredunant.shortheaders.fa
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 54418 sequences in 21.3623 seconds.




In [66]:
%%bash -s "$evoDir"

module load BLAST+/2.12.0-Linux_x86_64

## testis-expressed vs MACACA
blastp -query /projects_eg/projects/marta/TestisRestricted_Microproteins_TSA/with_TranscriptomeReconstruction/ribORF_humanTestis_in2.fa \
-db $1/macaca_db_prot \
-task 'blastp-short' \
-out $1/TestisExpressedHuman_vs_macaca.txt \
-evalue 0.0001 \
-outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qlen slen qframe sframe sstrand qcovs"

## testis-expressed vs MOUSE
blastp -query /projects_eg/projects/marta/TestisRestricted_Microproteins_TSA/with_TranscriptomeReconstruction/ribORF_humanTestis_in2.fa \
-db $1/mouse_db_prot \
-task 'blastp-short' \
-out $1/TestisExpressedHuman_vs_mouse.txt \
-evalue 0.0001 \
-outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qlen slen qframe sframe sstrand qcovs"

## Analysis

In [22]:
ribORF_in_2 = pd.read_csv("/projects_eg/projects/marta/TestisRestricted_Microproteins_TSA/with_TranscriptomeReconstruction/ribORF_humanTestis_in2.csv")
ribORF_in_2

,orfID,ORFseq,gene_id,gene_name,transcript_id,gene_type,orfType,length,length_aa,start_codon,ORFpep
0,ENST00000000412.8:12:-|10|2450:160:994|canonic...,ATGTTCCCTTTCTACAGCTGCTGGAGGACTGGACTGCTACTACTAC...,ENSG00000003056,M6PR,ENST00000000412,protein_coding,canonical,834,278,ATG,MFPFYSCWRTGLLLLLLAVAVRESWQTEEKTCDLVGEKGKESEKEL...
1,ENST00000001008.6:12:+|5|3715:171:1551|canonic...,ATGACAGCCGAGGAGATGAAGGCGACCGAGAGCGGGGCGCAGTCGG...,ENSG00000004478,FKBP4,ENST00000001008,protein_coding,canonical,1380,460,ATG,MTAEEMKATESGAQSAPLPMEGVDISPKQDEGVLKVIKREGTGTEM...
2,ENST00000001146.7:2:-|1|4556:29:1568|canonical...,ATGCTCTTTGAGGGCTTGGATCTGGTGTCGGCGCTGGCCACCCTCG...,ENSG00000003137,CYP26B1,ENST00000001146,protein_coding,canonical,1539,513,ATG,MLFEGLDLVSALATLAACLVSVTLLLAVSQQLWQLRWAATRDKSCK...
3,ENST00000002165.11:6:-|3|2385:87:1491|canonica...,ATGCGGCCCCAGGAGCTCCCCAGGCTCGCGTTCCCGTTGCTGCTGT...,ENSG00000001036,FUCA2,ENST00000002165,protein_coding,canonical,1404,468,ATG,MRPQELPRLAFPLLLLLLLLLPPPPCPAHSATRFDPTWESLDARQL...
4,ENST00000002596.6:4:-|24|7160:305:1229|canonic...,ATGGCCGCGCTGCTCCTGGGCGCGGTGCTGCTGGTGGCCCAGCCCC...,ENSG00000002587,HS3ST1,ENST00000002596,protein_coding,canonical,924,308,ATG,MAALLLGAVLLVAQPQLVPSRPAELGQQELLRKAGTLQDDVRDGVA...
...,...,...,...,...,...,...,...,...,...,...,...
12861,TCONS_00001617:4:-|1|314:11:257|noncoding|CTG,CTGCGCGCTTCCGCGGCAGCCGCGGCGGCCGAGCTGAGTGTGCGGG...,XLOC_001342,XLOC_001342,TCONS_00001617,novel,noncoding,246,82,CTG,MRASAAAAAAELSVRARRAKRVGLRRRAGKGGWDVVPALATLQRLA...
12862,TCONS_00001635:5:-|10|1280:135:180|noncoding|ATG,ATGATGACGTCGGCCAGTAACAGCAGAAGTGAAAATATGTGGTAG,XLOC_001408,XLOC_001408,TCONS_00001635,novel,noncoding,45,15,ATG,MMTSASNSRSENMW*
12863,TCONS_00001784:6:-|13|690:130:331|noncoding|ATG,ATGAGGGCAACGACGAGACCGTACAGGCCGAGCACCTCGGGGAAGA...,XLOC_001498,XLOC_001498,TCONS_00001784,novel,noncoding,201,67,ATG,MRATTRPYRPSTSGKMRIRIMPMNSRGCWAVARTPASPKMSMAKPA...
12864,TCONS_00001992:8:+|9|1191:153:675|noncoding|ATG,ATGACCGGGTCAGCGGAAATTTCAGAGCCAGCCAGCAGCTCAGGTC...,XLOC_001676,XLOC_001676,TCONS_00001992,novel,noncoding,522,174,ATG,MTGSAEISEPASSSGPRGRGRNGTAPRRQPRGHSRATADSGTSLSA...


In [31]:
## Analysis in Mouse
human_mouse = pd.read_csv(os.path.join(evoDir,"TestisExpressedHuman_vs_mouse.txt"), sep="\t", header=None)
human_mouse.columns = ["qseqid","sseqid","pident","length","mismatch","gapopen","qstart","qend","sstart","send","evalue","bitscore","qlen","slen","qframe","sframe","sstrand","qcovs"]

ribORF_in_2['mouse'] = np.where(ribORF_in_2['orfID'].isin(human_mouse.qseqid.values.tolist()), "conserved_mouse","no")
ribORF_in_2.groupby(['gene_type','mouse']).count()


                                      orfID  ORFseq  gene_id  gene_name  \
gene_type            mouse                                                
lncRNA               conserved_mouse     95      95       95         95   
                     no                 342     342      342        342   
novel                conserved_mouse      1       1        1          1   
                     no                  52      52       52         52   
processed_pseudogene conserved_mouse    974     974      974        974   
                     no                 135     135      135        135   
protein_coding       conserved_mouse  11076   11076    11076      11076   
                     no                 191     191      191        191   

                                      transcript_id  orfType  length  \
gene_type            mouse                                             
lncRNA               conserved_mouse             95       95      95   
                     no          

,orfID,ORFseq,gene_id,gene_name,transcript_id,gene_type,orfType,length,length_aa,start_codon,ORFpep,macaca,mouse
0,ENST00000000412.8:12:-|10|2450:160:994|canonic...,ATGTTCCCTTTCTACAGCTGCTGGAGGACTGGACTGCTACTACTAC...,ENSG00000003056,M6PR,ENST00000000412,protein_coding,canonical,834,278,ATG,MFPFYSCWRTGLLLLLLAVAVRESWQTEEKTCDLVGEKGKESEKEL...,conserved_macaca,conserved_mouse
1,ENST00000001008.6:12:+|5|3715:171:1551|canonic...,ATGACAGCCGAGGAGATGAAGGCGACCGAGAGCGGGGCGCAGTCGG...,ENSG00000004478,FKBP4,ENST00000001008,protein_coding,canonical,1380,460,ATG,MTAEEMKATESGAQSAPLPMEGVDISPKQDEGVLKVIKREGTGTEM...,conserved_macaca,conserved_mouse
2,ENST00000001146.7:2:-|1|4556:29:1568|canonical...,ATGCTCTTTGAGGGCTTGGATCTGGTGTCGGCGCTGGCCACCCTCG...,ENSG00000003137,CYP26B1,ENST00000001146,protein_coding,canonical,1539,513,ATG,MLFEGLDLVSALATLAACLVSVTLLLAVSQQLWQLRWAATRDKSCK...,conserved_macaca,conserved_mouse
3,ENST00000002165.11:6:-|3|2385:87:1491|canonica...,ATGCGGCCCCAGGAGCTCCCCAGGCTCGCGTTCCCGTTGCTGCTGT...,ENSG00000001036,FUCA2,ENST00000002165,protein_coding,canonical,1404,468,ATG,MRPQELPRLAFPLLLLLLLLLPPPPCPAHSATRFDPTWESLDARQL...,conserved_macaca,conserved_mouse
4,ENST00000002596.6:4:-|24|7160:305:1229|canonic...,ATGGCCGCGCTGCTCCTGGGCGCGGTGCTGCTGGTGGCCCAGCCCC...,ENSG00000002587,HS3ST1,ENST00000002596,protein_coding,canonical,924,308,ATG,MAALLLGAVLLVAQPQLVPSRPAELGQQELLRKAGTLQDDVRDGVA...,conserved_macaca,conserved_mouse
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12861,TCONS_00001617:4:-|1|314:11:257|noncoding|CTG,CTGCGCGCTTCCGCGGCAGCCGCGGCGGCCGAGCTGAGTGTGCGGG...,XLOC_001342,XLOC_001342,TCONS_00001617,novel,noncoding,246,82,CTG,MRASAAAAAAELSVRARRAKRVGLRRRAGKGGWDVVPALATLQRLA...,no,no
12862,TCONS_00001635:5:-|10|1280:135:180|noncoding|ATG,ATGATGACGTCGGCCAGTAACAGCAGAAGTGAAAATATGTGGTAG,XLOC_001408,XLOC_001408,TCONS_00001635,novel,noncoding,45,15,ATG,MMTSASNSRSENMW*,no,no
12863,TCONS_00001784:6:-|13|690:130:331|noncoding|ATG,ATGAGGGCAACGACGAGACCGTACAGGCCGAGCACCTCGGGGAAGA...,XLOC_001498,XLOC_001498,TCONS_00001784,novel,noncoding,201,67,ATG,MRATTRPYRPSTSGKMRIRIMPMNSRGCWAVARTPASPKMSMAKPA...,no,no
12864,TCONS_00001992:8:+|9|1191:153:675|noncoding|ATG,ATGACCGGGTCAGCGGAAATTTCAGAGCCAGCCAGCAGCTCAGGTC...,XLOC_001676,XLOC_001676,TCONS_00001992,novel,noncoding,522,174,ATG,MTGSAEISEPASSSGPRGRGRNGTAPRRQPRGHSRATADSGTSLSA...,no,no


In [36]:
## Analysis in Macaca
human_macaca = pd.read_csv(os.path.join(evoDir,"TestisExpressedHuman_vs_macaca.txt"), sep="\t", header=None)
human_macaca.columns = ["qseqid","sseqid","pident","length","mismatch","gapopen","qstart","qend","sstart","send","evalue","bitscore","qlen","slen","qframe","sframe","sstrand","qcovs"]

ribORF_in_2['macaca'] = np.where(ribORF_in_2['orfID'].isin(human_macaca.qseqid.values.tolist()), "conserved_macaca","no")
# ribORF_in_2_macaca.drop(columns=['mouse'], inplace=True)
ribORF_in_2.to_csv(os.path.join(evoDir,"ribORF_humanTestis_in2.evolution.csv"), index=None)
ribORF_in_2.groupby(['gene_type','macaca','mouse']).count()


orfID  ORFseq  gene_id  \
gene_type            macaca           mouse                                     
lncRNA               conserved_macaca conserved_mouse     94      94       94   
                                      no                  57      57       57   
                     no               conserved_mouse      1       1        1   
                                      no                 285     285      285   
novel                conserved_macaca conserved_mouse      1       1        1   
                                      no                   7       7        7   
                     no               no                  45      45       45   
processed_pseudogene conserved_macaca conserved_mouse    928     928      928   
                                      no                  72      72       72   
                     no               conserved_mouse     46      46       46   
                                      no                  63      63       63   
protein_coding       conserved_macaca conserved_mouse  10920   10920    10920   
                                      no                 168     168      168   
                     no               conserved_mouse    156     156      156   
                                      no                  23      23       23   

                                                       gene_name  \
gene_type            macaca           mouse                        
lncRNA               conserved_macaca conserved_mouse         94   
                                      no                      57   
                     no               conserved_mouse          1   
                                      no                     285   
novel                conserved_macaca conserved_mouse          1   
                                      no                       7   
                     no               no                      45   
processed_pseudogene conserved_macaca conserved_mouse        928   
                                      no                      72   
                     no               conserved_mouse         46   
                                      no                      63   
protein_coding       conserved_macaca conserved_mouse      10920   
                                      no                     168   
                     no               conserved_mouse        156   
                                      no                      23   

                                                       transcript_id  orfType  \
gene_type            macaca           mouse                                     
lncRNA               conserved_macaca conserved_mouse             94       94   
                                      no                          57       57   
                     no               conserved_mouse              1        1   
                                      no                         285      285   
novel                conserved_macaca conserved_mouse              1        1   
                                      no                           7        7   
                     no               no                          45       45   
processed_pseudogene conserved_macaca conserved_mouse            928      928   
                                      no                          72       72   
                     no               conserved_mouse             46       46   
                                      no                          63       63   
protein_coding       conserved_macaca conserved_mouse          10920    10920   
                                      no                         168      168   
                     no               conserved_mouse            156      156   
                                      no                          23       23   

                                                       length  length_aa  \
gene_type            macaca           mouse                       